In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\tejas\anaconda3\envs\medchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY="c7a12670-71bb-46f6-a96f-e210732babbe"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents        

In [5]:
extracted_data=load_pdf("data/")

In [6]:
extracted_data

[Document(page_content='Resources\nBOOKS\nA Manual of Laboratory and Diagnostic Tests. 5th ed. Ed.\nFrancis Fishback. Philadelphia: Lippincott, 1996.\nGarza, Diana, and Kathleen Becan-McBride. Phlebotomy\nHandbook. 4th ed. Stamford: Appleton & Lange, 1996.\nPERIODICALS\nMoul, Judd W., et al. “The Contemporary Value of Pretreat-\nment Prostatic Acid Phosphatase to Predict PathologicalStage and Recurrence in Radical Prostatectomy Cases.”Journal of Urology (Mar. 1998): 935-940.\nNancy J. Nordenson\nAcid reflux seeHeartburn\nAcidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 millionpeople in the United States. While acne can arise at anyage, it usually be

In [7]:
#create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:",len(text_chunks))

length of my chunk: 32


In [9]:
text_chunks

[Document(page_content='Resources\nBOOKS\nA Manual of Laboratory and Diagnostic Tests. 5th ed. Ed.\nFrancis Fishback. Philadelphia: Lippincott, 1996.\nGarza, Diana, and Kathleen Becan-McBride. Phlebotomy\nHandbook. 4th ed. Stamford: Appleton & Lange, 1996.\nPERIODICALS\nMoul, Judd W., et al. “The Contemporary Value of Pretreat-\nment Prostatic Acid Phosphatase to Predict PathologicalStage and Recurrence in Radical Prostatectomy Cases.”Journal of Urology (Mar. 1998): 935-940.\nNancy J. Nordenson\nAcid reflux seeHeartburn', metadata={'source': 'data\\Encyclopedia of Medicine-38-40.pdf', 'page': 0}),
 Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is', metadata={'source': 'data\\En

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings=download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
#testing embedding model
query_result=embeddings.embed_query("Hi! how are you?")
print("Length =",len(query_result))

Length = 384


In [14]:
import os
from langchain_pinecone import PineconeVectorStore

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

index_name = "medical-chatbot"

# Connect to Pinecone index and insert the chunked docs as contents
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [30]:
from pinecone import Pinecone
pinecone_obj = Pinecone(api_key=PINECONE_API_KEY)  

 
index_name="medical-chatbot"
index = pinecone_obj.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7031}},
 'total_vector_count': 7031}

In [15]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What is acne?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is', metadata={'page': 0.0, 'source': 'data\\Encyclopedia of Medicine-38-40.pdf'}), Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is', metadata={'page': 0.0, 'source': 'data\\Encyclopedia of Medicine-38-40.pdf'}), Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. A

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])